In [1]:
def implicit():
    from google.cloud import storage

    # If you don't specify credentials when constructing the client, the
    # client library will look for credentials in the environment.
    storage_client = storage.Client()

    # Make an authenticated API request
    buckets = list(storage_client.list_buckets())
    print(buckets)

In [2]:
def run_quickstart(file_name):
    import io
    import os
 
    # 구글 라이브러리 import
    from google.cloud import vision
    from google.cloud.vision_v1 import types
 
    # 사용할 클라이언트 설정
    client = vision.ImageAnnotatorClient()
    
    # 이미지 읽기
    with io.open(file_name, 'rb') as image_file:
        content = image_file.read()
 
    image = types.Image(content=content)
 
    # label 뽑아냄.
    response = client.label_detection(image=image)
    labels = response.label_annotations

    ## 물체 처리 과정
    flag=0
    print('1) 물건 종류')
    for label in labels:
        print('\t\t',label.description)
        if "Metal" in label.description.split() or "Glass" in label.description.split() or "Cup" in label.description.split() or "equipment" in label.description.split():
            flag=1
    return flag

In [3]:
from scipy.spatial.distance import euclidean
from imutils import perspective
from imutils import contours
import numpy as np
import imutils
import cv2

#이미지 경로


def detectsize(img_path):
    #이미지 읽기
    image = cv2.imread(img_path)
    #이미지 전처리
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #흑백으로 바꿈
    blur = cv2.GaussianBlur(gray, (9, 9), 0)#가우시안 필터

    edged = cv2.Canny(blur, 50, 100)
    edged = cv2.dilate(edged, None, iterations=1)
    edged = cv2.erode(edged, None, iterations=1)

    #show_images([blur, edged]) #전처리된 영상

    # 윤각 찾기
    cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)

    #가장 왼쪽 기준 객체부터 왼->오른쪽으로 윤각선 정렬
    (cnts, _) = contours.sort_contours(cnts)

    # 충분히 크지않은 물체는 무시
    cnts = [x for x in cnts if cv2.contourArea(x) > 75]

    #cv2.drawContours(image, cnts, -1, (0,255,0), 3) #물체 테두리

    #show_images([image, edged]) #경계선
    #print(len(cnts)) #배열길이= 검출된 덩어리 갯수

    #기준 도형 (여기선 2cm X 2cm 정사각형을 기준으로함)
    ref_object = cnts[0]
    box = cv2.minAreaRect(ref_object)
    box = cv2.boxPoints(box)
    box = np.array(box, dtype="int")
    box = perspective.order_points(box)
    (tl, tr, br, bl) = box
    dist_in_pixel = euclidean(tl, tr) #검출한 픽셀크기
    dist_in_cm = 2 #기준 물체의 cm
    pixel_per_cm = dist_in_pixel/dist_in_cm  #검출한 픽셀/실제 cm로 거리에 따른 비율 구함

    widli=[]
    htli=[]

    flag=0

    # 경계선 그리기
    for cnt in cnts:
        box = cv2.minAreaRect(cnt)
        box = cv2.boxPoints(box)
        box = np.array(box, dtype="int")
        box = perspective.order_points(box)
        (tl, tr, br, bl) = box

        cv2.drawContours(image, [box.astype("int")], -1, (0, 0, 255), 2)
        mid_pt_horizontal = (tl[0] + int(abs(tr[0] - tl[0])/2), tl[1] + int(abs(tr[1] - tl[1])/2))
        mid_pt_verticle = (tr[0] + int(abs(tr[0] - br[0])/2), tr[1] + int(abs(tr[1] - br[1])/2))

        wid = euclidean(tl, tr)/pixel_per_cm #가로 비율로 계산
        ht = euclidean(tr, br)/pixel_per_cm #세로 비율로 계산
        
        #세로 프린트
        widli.append(wid)
        htli.append(ht)
        #print(widli)
        #print(htli)
        
        #print("{:.1f}cm".format(wid)), (int(mid_pt_horizontal[0] - 15), int(mid_pt_horizontal[1] - 10))
        #가로 프린트
        #print("{:.1f}cm".format(ht))#, (int(mid_pt_verticle[0] - 15), int(mid_pt_verticle[1])))
        #print(widli)
        #print(htli)
    return widli,htli

In [4]:
img_path= "camera1.jpg"
widli,htli=detectsize(img_path)
x=widli[1]
y=htli[1]
flag=run_quickstart(img_path)
frag=""
print("2) 물건 특징")
if flag==1:
    print("\t\t깨지기쉬운 , 다룰 때 주의가 필요한")
    frag="충전재가 포함된"
else:
    print("\t\tNone")

img_path="camera2.jpg"
widli,htli=detectsize(img_path)
z=htli[1]

print('3) 물건 사이즈')
print("\t\t",x,"*",y,"*",z,"cm")
x+=5
y+=5
z+=5
x=int(x)
y=int(y)
z=int(z)
if flag==1:
    x+=10
print("\n\n====따라서 ",x,"*",y,"*",z,"cm 의 ",frag,"종이박스를 추천합니다!====")
#print(htli)
#print(widli)
#print(flag)


1) 물건 종류
		 Automotive lighting
		 Helmet
		 Automotive tire
		 Automotive design
		 Tool
		 Font
		 Auto part
		 Bumper
		 Automotive exterior
		 Personal protective equipment
2) 물건 특징
		깨지기쉬운 , 다룰 때 주의가 필요한
3) 물건 사이즈
		 13.506598451207877 * 6.772699518985228 * 1.1093467964278465 cm


====따라서  28 * 11 * 6 cm 의  충전재가 포함된 종이박스를 추천합니다!====


In [5]:
img_path= "mouse2.jpg"
widli,htli=detectsize(img_path)
x=widli[1]
y=htli[1]
flag=run_quickstart(img_path)
frag=""
print("2) 물건 특징")
if flag==1:
    print("\t\t깨지기쉬운 , 다룰 때 주의가 필요한")
    frag="충전재가 포함된"
else:
    print("\t\tNone")

img_path="mouse1.jpg"
widli,htli=detectsize(img_path)
z=htli[1]

print('3) 물건 사이즈')
print("\t\t",x,"*",y,"*",z,"cm")
x+=5
y+=5
z+=5
x=int(x)
y=int(y)
z=int(z)
if flag==1:
    x+=10
print("\n\n====따라서 ",x,"*",y,"*",z,"cm 의 ",frag,"종이박스를 추천합니다!====")
#print(htli)
#print(widli)
#print(flag)

1) 물건 종류
		 Peripheral
		 Input device
		 Mouse
		 Gadget
		 Font
		 Computer hardware
		 Computer component
		 Logo
		 Circle
		 Rectangle
2) 물건 특징
		None
3) 물건 사이즈
		 8.666666666666666 * 4.794871794871795 * 2.5513834193132925 cm


====따라서  13 * 9 * 7 cm 의   종이박스를 추천합니다!====
